In [1]:
import psycopg2
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [5]:
# Connect to PostgreSQL and load the data
conn = psycopg2.connect(
    host = "localhost", 
    dbname = "mnrj", 
    user= "postgres", 
    password = "gwdb", 
    port = 5600
)

# Query the table and load it into a DataFrame
query = "SELECT * FROM nba;"
data = pd.read_sql(query, conn)
conn.close()

<positron-console-cell-5>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


In [7]:
from transformers import AutoModel, AutoTokenizer

In [8]:
target_columns = ['pts', 'trb', 'ast', 'p_r_a', 'p_r', 'p_a', 'a_r']
print( [col for col in data.columns ])

['id', 'player', 'date', 'age', 'team', 'hoa', 'opp', 'gs', 'mp', 'fg', 'fga', 'fg_percent', 'twop', 'twop_percent', 'threep', 'ft', 'ft_percent', 'ts_percent', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'gmsc', 'bpm', 'plus_minus', 'pos', 'player_additional', 'result', 'total_score', 'p_r_a', 'p_r', 'p_a', 'a_r', 'month', 'days_since', 'teammates_rebounds', 'teammates_assists', 'teammates_pr', 'teammates_pa', 'teammates_ar', 'teammates_pra', 'opponents_rebounds', 'opponents_assists', 'opponents_pr', 'opponents_pa', 'opponents_ar', 'opponents_pra']


In [9]:
print(data.head)

<bound method NDFrame.head of          id        player  ...                                       opponents_ar                                      opponents_pra
0      6230  Rayan Rupert  ...  [{'ar': 15, 'player': 'Stephen Curry'}, {'ar':...  [{'pra': 37, 'player': 'Stephen Curry'}, {'pra...
1      6231  Rayan Rupert  ...  [{'ar': 21, 'player': 'Amen Thompson'}, {'ar':...  [{'pra': 38, 'player': 'Amen Thompson'}, {'pra...
2      6232  Rayan Rupert  ...  [{'ar': 8, 'player': 'Harrison Barnes'}, {'ar'...  [{'pra': 25, 'player': 'Harrison Barnes'}, {'p...
3      6233  Terry Rozier  ...  [{'ar': 10, 'player': 'Trae Young'}, {'ar': 8,...  [{'pra': 33, 'player': 'Trae Young'}, {'pra': ...
4      6234  Terry Rozier  ...  [{'ar': 8, 'player': 'Isaiah Stewart'}, {'ar':...  [{'pra': 23, 'player': 'Isaiah Stewart'}, {'pr...
...     ...           ...  ...                                                ...                                                ...
30596  6225  Rayan Rupert  ...  [{'ar':